In [1]:
import io
import os
import base64
import requests
from PIL import Image
from io import BytesIO
from datetime import datetime
from google.cloud import storage
from google.cloud import aiplatform
from vertexai.language_models import TextGenerationModel

In [2]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

In [3]:
GCS_BUCKET = "github-repo"
!gsutil ls gs://$GCS_BUCKET/product_img/

gs://github-repo/product_img/
gs://github-repo/product_img/fashion1.jpeg
gs://github-repo/product_img/fashion2.jpeg
gs://github-repo/product_img/fashion3.jpeg
gs://github-repo/product_img/fashion4.jpeg
gs://github-repo/product_img/fashion5.jpeg
gs://github-repo/product_img/fashion6.jpeg
gs://github-repo/product_img/fashion7.jpeg
gs://github-repo/product_img/fashion8.jpeg


In [4]:

SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai-restricted/vertex-vision-model-garden-dockers/pytorch-transformers-serve"



In [5]:
def create_job_name(prefix):
    user = os.environ.get("USER")
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    job_name = f"{prefix}-{user}-{now}"
    return job_name


def download_image(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content))


def image_to_base64(image, format="JPEG"):
    buffer = BytesIO()
    image.save(buffer, format=format)
    image_str = base64.b64encode(buffer.getvalue()).decode("utf-8")
    return image_str


def base64_to_image(image_str):
    image = Image.open(BytesIO(base64.b64decode(image_str)))
    return image


def image_grid(imgs, rows=2, cols=2):
    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid


def deploy_model(model_id, task):
    model_name = "blip-image-captioning"
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")
    serving_env = {
        "MODEL_ID": model_id,
        "TASK": task,
    }
    # If the model_id is a GCS path, use artifact_uri to pass it to serving docker.
    artifact_uri = model_id if model_id.startswith("gs://") else None
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/transformers_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
        artifact_uri=artifact_uri,
    )
    model.deploy(
        endpoint=endpoint,
        machine_type="n1-standard-8",
        accelerator_type="NVIDIA_TESLA_T4",
        accelerator_count=1,
        deploy_request_timeout=1800,
    )
    return model, endpoint


def read_jpeg_image_from_gcs(bucket_name, image_name):
    """Reads a JPEG image from a Google Cloud Storage (GCS) bucket.

    Args:
    bucket_name: The name of the GCS bucket that contains the image file.
    image_name: The name of the image file in the GCS bucket.

    Returns:
    The image file as a PIL Image object.
    """

    # Import the Google Cloud Storage client library.

    # Create a storage client.
    client = storage.Client()

    # Get the bucket object.
    bucket = client.bucket(bucket_name)

    # Get the blob object.
    blob = bucket.blob(image_name)

    # Read the blob to a bytestring.
    image_data = blob.download_as_bytes()

    # Decode the bytestring to a PIL Image object.
    image = Image.open(io.BytesIO(image_data))

    # Return the PIL Image object.
    return image

In [ ]:
model, endpoint = deploy_model(
    model_id="Salesforce/blip-image-captioning-base", task="image-to-text"
)

Creating Endpoint
Create Endpoint backing LRO: projects/1090925531874/locations/us-central1/endpoints/3755019125831761920/operations/2906612520498233344
Endpoint created. Resource name: projects/1090925531874/locations/us-central1/endpoints/3755019125831761920
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/1090925531874/locations/us-central1/endpoints/3755019125831761920')
Creating Model
Create Model backing LRO: projects/1090925531874/locations/us-central1/models/7469648791528800256/operations/8698241641296691200
Model created. Resource name: projects/1090925531874/locations/us-central1/models/7469648791528800256@1
To use this Model in another session:
model = aiplatform.Model('projects/1090925531874/locations/us-central1/models/7469648791528800256@1')
Deploying model to Endpoint : projects/1090925531874/locations/us-central1/endpoints/3755019125831761920
Deploy Endpoint model backing LRO: projects/1090925531874/locations/us-central1/endpoints/375501

In [ ]:
def generate_product_description(model, image_caption, temperature=0):
    """Ideation example with a Large Language Model"""
    prompt_prefix = "Imagine you are a digital marketer working for a retail organization. \
                    You are an expert in building detailed and catchy descriptions fro the retail fashion products on your website.\
                    Generate a product description using the following short caption that describes the apparel"
    prompt = prompt_prefix + image_caption
    response = model.predict(
        prompt,
        temperature=temperature,
        max_output_tokens=256,
        top_k=40,
        top_p=0.8,
    )
    return response.text

In [ ]:
from matplotlib import pyplot as plt

for i in range(1, 9):
    image_data = read_jpeg_image_from_gcs(
        GCS_BUCKET, "product_img/fashion" + str(i) + ".jpeg"
    )
    # Display the image
    plt.imshow(image_data)
    plt.axis("off")
    plt.show()

    instances = [
        {"image": image_to_base64(image_data)},
    ]
    preds = endpoint.predict(instances=instances).predictions
    print(preds)
    product_description = generate_product_description(
        model=generation_model, image_caption=preds[0]
    )
    print(product_description)